In [1]:
import numpy as np 
import theano 
import theano.tensor as T
from theano import pp 
import sys
sys.path.append('/home/kman/papers/QFPaper/Code/')
from copulas import * 
from hmm import * 

from scipy.stats import expon 
from scipy.stats import norm 

theta = 1.5
myCopula = Gumbel(theta) 

rng = np.random.RandomState(123)

#sample some exponential and normal random numbers and get the grades
n_samples = 10000 
ll = 2 
sigma = 1.5 
taus = rng.exponential(scale=1./ll, size=n_samples) 
pcs = rng.normal(0., sigma, size=n_samples) 
us = expon.cdf(taus, scale=1./ll) 
vs = norm.cdf(pcs, scale=sigma) 


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
t = T.scalar('t') 
tt = T.vector('tt')
theta_ = T.scalar('theta')
phi = (-T.log(t))**theta_ 
dphi = T.grad(phi,t)
ddphi = T.grad(dphi, t)
dddphi = T.grad(ddphi, t) 

dphi_f = theano.function([t, theta_], dphi) 
ddphi_f = theano.function([t, theta_], ddphi) 
dddphi_f = theano.function([t, theta_], dddphi) 

theano_results = []
copula_module_results =[] 

def psi_and_derivs_theano(u, v): 
    
    c_u_v = myCopula.psi_inv(myCopula.psi(u) + myCopula.psi(v)) 
    
    return [ 
            c_u_v,
            float(dphi_f(u, theta)), 
            float(dphi_f(v, theta)),
            float(dphi_f(c_u_v, theta)), 
            float(ddphi_f(u, theta)),
            float(ddphi_f(v, theta)),
            float(ddphi_f(c_u_v, theta)),
            float(dddphi_f(c_u_v, theta))
           ]

for u, v in zip(us, vs):
    theano_results.append(psi_and_derivs_theano(u,v)) 

for u, v in zip(us, vs): 
    copula_module_results.append(myCopula.psi_and_derivs(u, v)) 
    
# for i, cr in enumerate(copula_module_results): 
#      ll = np.asarray(cr) - np.asarray(theano_results[i])#  )/np.asarray(cr)
#      if abs(ll.sum()) > 1e-6: 
#          print i, copula_module_results[i][0]

        

In [ ]:
from numpy.testing import assert_array_almost_equal 

theano_results = np.asarray(theano_results)
copula_module_results = np.asarray(copula_module_results)
# uncomment this to see failures when the argument of the calling function
#i.e. c(u,v) is of O(1e-4)

#assert_array_almost_equal(copula_module_results, theano_results, 
#                          decimal=6)

# or use absolute tolerance here to see failures when the argument to the 3rd derivative is of O(1e-4)  
#np.allclose(theano_results, copula_module_results, rtol=1e-06)

In [ ]:
import numpy as np 
import pdb
def ff(t): 
    return theta*((-np.log(t))**(theta-1) + (theta-1)*(-np.log(t))**(theta-2))/t**2
    
def dff(t): 
    m_log_t = -np.log(t)
    #pdb.set_trace()
    return (-2.*theta*m_log_t**(theta-1) - 3.*theta*(theta-1)*m_log_t**(theta-2)\
     -theta*(theta-1)*(theta-2)*m_log_t**(theta-3))/t**3

t = theano_results[844][0] 
epsilon = 0.001 * t 

dff_hat = (ff(t+epsilon) - ff(t-epsilon))/(2.*epsilon)

dff(t), dddphi_f(t, theta) 





# Test here the copula partial derivs 


In [5]:
var_u = T.scalar('u') 
var_v = T.scalar('v') 

# redefine the copula here
phi_sum = T.power(-T.log(var_u), theta_) + T.power(-T.log(var_v), theta_)

Copula = T.exp(-T.power(phi_sum, 1/theta_)) 
Copula_f = theano.function([var_u, var_v, theta_], phi_sum)

# and calculate the relevant derivatives using Theano 
dCopula_du = T.grad(Copula, var_u) 
ddCopula_dudv = T.grad(dCopula_du, var_v) 
cop_pdf = ddCopula_dudv  
cop_pdf_f = theano.function([var_u,var_v, theta_], cop_pdf)

dcop_pdf_du = T.grad(ddCopula_dudv, var_u)
dcop_pdf_dv = T.grad(cop_pdf, var_v)

dcop_pdf_du_f = theano.function([var_u,var_v,theta_], dcop_pdf_du) 
dcop_pdf_dv_f = theano.function([var_u,var_v,theta_], dcop_pdf_dv) 


# generate a row with all the generator derivatives that are consumed 
# copula_dens_and_derivs. Lets do this with Theano here. 
u = us[0]; v = vs[0] 
C = Copula_f(u,v, theta)
row = {} 
dphi_f(u,theta) 
row['psi_prime_u']= float(dphi_f(u,theta))         
row['psi_prime_v']= float(dphi_f(v,theta))
row['psi_prime_C']= float(dphi_f(C,theta))
                         
row['psi_twoprime_u']= float(ddphi_f(u, theta))
row['psi_twoprime_v']= float(ddphi_f(v, theta))
row['psi_twoprime_C']= float(ddphi_f(C, theta))
row['psi_threeprime_C']= float(dddphi_f(C, theta))

gumbel_results = np.asarray(Gumbel.copula_dens_and_derivs(row))

theano_results = np.asarray(
                 [
                  cop_pdf_f(u,v, theta),
                  dcop_pdf_du_f(u,v,theta), 
                  dcop_pdf_dv_f(u,v, theta)
                 ])
gumbel_results - theano_results 
u, v

(0.69646918559786164, 0.83106967897697204)